In [8]:
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Joining ookla data to census blocks

We want everything pulled into the census blocks because that's what the FCC data level is 

Ok so for each census block we want the following info:
- max/min/mean download
- max/min/mean upload

I think what we want to do is batch this shit. So first, we sjoin Ookla fixed data with states data so each Ookla point is associated with a state. Then, we cycle through our census block chunks, segmenting the Ookla data as we go and just doing sjoins with Ookla data that are in the census block state groupings. 

For example, for block file `tl_2019_01to09_tabblock10.shp` would first segment the ookla data to just include squares from states with fips codes 01 - 09. Then it would do the spatial join between `tl_2019_01to09_tabblock10.shp` and `ookla-fixed-01to09.shp`.

So we'll want to... probably what we have to do is join the blocks onto the ookla rather than the ookla to the blocks. Then we can perform the aggregates on the ookla data so we get those max/min/means. Then we'll need to join those back onto the shapefile blocks so we can map. 

## Join Ookla fixed data to states shapefile

In [4]:
states = gpd.read_file('../../../GIS/tl_2019_us_state/tl_2019_us_state.shp')

In [5]:
states.columns

Index(['REGION', 'DIVISION', 'STATEFP', 'STATENS', 'GEOID', 'STUSPS', 'NAME',
       'LSAD', 'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'geometry'],
      dtype='object')

In [6]:
ookla_fixed = gpd.read_file('../data/ookla/2020-01-01_performance_fixed_tiles/gps_fixed_tiles.shp')

In [7]:
len(ookla_fixed)

6688699

In [8]:
ookla_fixed.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry
0,3100121113233332,16599,13312,70,37,27,"POLYGON ((106.68823 -6.30984, 106.69373 -6.309..."
1,3112231300122103,42461,13516,19,171,53,"POLYGON ((145.04700 -37.82714, 145.05249 -37.8..."
2,0331102110022021,25765,15185,16,228,100,"POLYGON ((-9.13513 38.71123, -9.12964 38.71123..."
3,1202020213110310,79703,15579,18,199,32,"POLYGON ((1.35132 51.38550, 1.35681 51.38550, ..."
4,1231210133132220,27032,24622,23,174,55,"POLYGON ((75.84961 30.91165, 75.85510 30.91165..."


In [10]:
states = states.to_crs('EPSG:4326')
ookla_states = gpd.sjoin(ookla_fixed, states, how='left', op='intersects')

In [12]:
len(ookla_states)

6692769

This tells us that there were some Ookla tile on state boundary lines so they went for both states and duplicated quadkeys. I think this is fine since census blocks fall within state boundaries. If there is an ookla tile that counts for two states, it won't be counted in the same census block twice which is all that matters here.

Just don't think you can use this count to talk about how many ookla tiles were produced because that would be false.

In [15]:
us_ookla = ookla_states.loc[~(ookla_states['NAME'].isna())]

In [16]:
us_ookla.to_file('../data/ookla/2020-01-01_performance_fixed_tiles/gps_fixed_tiles_w_us_state.shp')

In [17]:
us_ookla.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,REGION,DIVISION,...,GEOID,STUSPS,NAME,LSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
5,0231301222133033,166797,55407,18,38,10,"POLYGON ((-98.11340 29.71191, -98.10791 29.711...",25.0,3,7,...,48,TX,Texas,00,G4000,A,6.766682e+11,1.899188e+10,+31.4347032,-099.2818238
7,0302222300132222,152459,113630,7,53,18,"POLYGON ((-88.33008 41.90637, -88.32458 41.906...",2.0,2,3,...,17,IL,Illinois,00,G4000,A,1.437799e+11,6.215724e+09,+40.1028754,-089.1526108
12,0320003131302013,117856,67623,13,65,15,"POLYGON ((-84.53430 38.09566, -84.52881 38.095...",33.0,3,6,...,21,KY,Kentucky,00,G4000,A,1.022822e+11,2.372611e+09,+37.5336844,-085.2929801
13,0231112330132012,134153,27922,26,44,11,"POLYGON ((-93.24097 37.05079, -93.23547 37.050...",45.0,2,4,...,29,MO,Missouri,00,G4000,A,1.780499e+11,2.489250e+09,+38.3507500,-092.4567826
15,0231310312232012,37596,80679,13,15,1,"POLYGON ((-93.41675 30.17837, -93.41125 30.178...",20.0,3,7,...,22,LA,Louisiana,00,G4000,A,1.118991e+11,2.375200e+10,+30.8634368,-091.7987173


In [18]:
# convert to Mbps for easier reading
us_ookla['avg_d_mbps'] = us_ookla['avg_d_kbps']/1000
us_ookla['avg_u_mbps'] = us_ookla['avg_u_kbps']/1000

/Users/akanik/.local/share/virtualenvs/broadband-cNLU4PFc/lib/python3.8/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [27]:
us_ookla.columns

Index(['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry', 'index_right', 'REGION', 'DIVISION', 'STATEFP', 'STATENS',
       'GEOID', 'STUSPS', 'NAME', 'LSAD', 'MTFCC', 'FUNCSTAT', 'ALAND',
       'AWATER', 'INTPTLAT', 'INTPTLON', 'avg_d_mbps', 'avg_u_mbps'],
      dtype='object')

## Join us_ookla to census blocks, in chuncks

In [31]:
def run_ookla_blocks_chunck(block_file,states_list,blocks_ookla_file):
    print('Reading in block chunk...')
    blocks = gpd.read_file(block_file)
    blocks = blocks.to_crs('EPSG:4326')
    print('Creating ookla chunk...')
    ookla_chunk = us_ookla.loc[us_ookla['STATEFP'].isin(states_list)]
    ookla_chunk = ookla_chunk[['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
                    'geometry', 'STATEFP', 'STATENS','avg_d_mbps', 'avg_u_mbps']]
    print('Doing spatial join...')
    ookla_blocks = gpd.sjoin(ookla_chunk,blocks, how='left', op='intersects')
    print('Grouping by blocks...')
    ookla_grouped = ookla_blocks.groupby('GEOID10').agg({'quadkey':'count','tests':'sum','devices':'sum',
                                                         'avg_d_mbps':['max','min','mean'],
                                                         'avg_u_mbps':['max','min','mean']}).reset_index()
    ookla_grouped.columns = [' '.join(col).strip() for col in ookla_grouped.columns.values]

    print('Joining ookla grouped data back to blocks...')
    blocks_ookla = blocks.merge(ookla_grouped,on='GEOID10',how='left')
    print('Writing blocks with ookla to file...')
    blocks_ookla.to_file(blocks_ookla_file)

In [35]:
#vars to change each run
block_file = '../GIS/tl_2019_01to09_tabblock10.shp'
states_list = ['01','02','03','04','05','06','07','08','09']
blocks_ookla_file = '../GIS/blocks_ookla/tl_2019_01to09_tabblock10_ookla.shp'

print('Reading in block chunk...')
blocks = gpd.read_file(block_file)
blocks = blocks.to_crs('EPSG:4326')
print('Creating ookla chunk...')
ookla_chunk = us_ookla.loc[us_ookla['STATEFP'].isin(states_list)]
ookla_chunk = ookla_chunk[['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
                'geometry', 'STATEFP', 'STATENS','avg_d_mbps', 'avg_u_mbps']]
print('Doing spatial join...')
ookla_blocks = gpd.sjoin(ookla_chunk,blocks, how='left', op='intersects')
print('Grouping by blocks...')
ookla_grouped = ookla_blocks.groupby('GEOID10').agg({'quadkey':'count','tests':'sum','devices':'sum',
                                                     'avg_d_mbps':['max','min','mean'],
                                                     'avg_u_mbps':['max','min','mean']}).reset_index()
ookla_grouped.columns = ['_'.join(col).strip() for col in ookla_grouped.columns.values]

print('Joining ookla grouped data back to blocks...')
blocks_ookla = blocks.merge(ookla_grouped,on='GEOID10',how='left')
print('Writing blocks with ookla to file...')
blocks_ookla.to_file(blocks_ookla_file)

Reading in block chunk...
Creating ookla chunk...
Doing spatial join...
Grouping by blocks...
Joining ookla grouped data back to blocks...
Writing blocks with ookla to file...


In [37]:
len(blocks)

1704220

In [38]:
len(blocks_ookla)

1704220

In [40]:
blocks_ookla.sort_values('quadkey count',ascending=False).head()

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYPE,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,quadkey count,tests sum,devices sum,avg_d_mbps max,avg_d_mbps min,avg_d_mbps mean,avg_u_mbps max,avg_u_mbps min,avg_u_mbps mean
455260,04,013,010102,3017,040130101023017,Block 3017,G5040,R,None,None,S,26184891,0,+33.8751366,-111.8654706,"POLYGON ((-111.89124 33.85728, -111.89124 33.8...",53.0,959.0,242.0,288.135,9.968,98.444642,28.660,2.969,16.526585
1598305,08,067,970704,2030,080679707042030,Block 2030,G5040,R,None,None,S,193229215,14407,+37.5514994,-107.8803170,"POLYGON ((-107.95178 37.56582, -107.95178 37.5...",46.0,515.0,167.0,137.621,0.495,32.948739,153.769,0.156,15.872783
283772,02,170,000300,1043,021700003001043,Block 1043,G5040,R,None,None,S,11185161,0,+61.6622358,-149.2574539,"POLYGON ((-149.29999 61.66491, -149.29958 61.6...",46.0,274.0,91.0,325.420,8.015,47.009043,102.451,1.656,14.693543
1304451,06,061,990000,0001,060619900000001,Block 0001,G5040,R,None,None,S,0,71934484,+39.1775084,-120.0865328,"POLYGON ((-120.16194 39.09760, -120.16127 39.1...",45.0,482.0,184.0,195.038,1.098,73.839911,23.180,0.174,9.892311
934979,06,065,045125,1026,060650451251026,Block 1026,G5040,R,None,None,S,22059324,0,+33.6922237,-116.3412399,"POLYGON ((-116.37344 33.70032, -116.37332 33.7...",44.0,606.0,244.0,221.523,0.078,86.254045,232.078,2.159,21.445659


### FCC provider data
https://www.fcc.gov/general/broadband-deployment-data-fcc-form-477

Looks like the nationwide file is corrupt. We'll need to cycle through the state files which is fine. But we need the fips code --> abbreviations to make pulling from online logical.

And for these data, we want to groupby block and aggregate the following info:
- concat list of providers
- count of providers
- max advertised up
- max advertised down

In [50]:
states_abb = states[['STATEFP','STUSPS']].set_index('STATEFP').to_dict('index')
# use like this: states_abb[fips_code_str]['STUSPS']

'WV'

In [9]:
dtypes = {'BlockCode':'str'}
ak_fcc_fixed_sat = pd.read_csv('https://transition.fcc.gov/form477/BroadbandData/Fixed/Dec19/Version%201/AK-Fixed-Dec2019.zip',compression='zip',dtype=dtypes)

In [12]:
ak_fcc_fixed_sat.sort_values('MaxAdDown',ascending=False).head()

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business
113228,32008182,59349,1568880,GCI Communication Corp.,GCI Communication Corp.,GCI Holdings LLC,130534,GCI Holdings LLC,AK,020200022022002,43,1,1000.0,50.0,1
94225,19674084,56126,1573427,Matanuska Telephone Association Inc.,MATANUSKA TELEPHONE,"Matanuska Telephone Association, Inc.",130791,"Matanuska Telephone Association, Inc.",AK,021700001022139,50,1,1000.0,100.0,1
110612,32005566,59349,1568880,GCI Communication Corp.,GCI Communication Corp.,GCI Holdings LLC,130534,GCI Holdings LLC,AK,020200002024008,43,1,1000.0,50.0,1
110613,32005567,59349,1568880,GCI Communication Corp.,GCI Communication Corp.,GCI Holdings LLC,130534,GCI Holdings LLC,AK,020200002024009,43,1,1000.0,50.0,1
110614,32005568,59349,1568880,GCI Communication Corp.,GCI Communication Corp.,GCI Holdings LLC,130534,GCI Holdings LLC,AK,020200002024010,43,1,1000.0,50.0,1


In [6]:
len('20200018021005')

14

In [14]:
ak_fcc_fixed_sat.groupby('BlockCode').agg({'DBAName':'sum','Provider_Id':'nunique','MaxAdDown':['max','min','mean'],'MaxAdUp':['max','min','mean']}).reset_index().sample(50)

BlockCode                                            DBAName  \
                                                                      sum   
29479  021800001003013         HughesNetViasat IncGCI Communication Corp.   
24101  021640001002200                                          HughesNet   
9055   020680001002372                   HughesNetGCI Communication Corp.   
37838  022400004002033  HughesNetSkycastersWhitestone Power & Communic...   
40849  022700001001277                                          HughesNet   
7427   020500001004117                                          HughesNet   
1482   020160001001377                                          HughesNet   
39214  022610002001060                      HughesNetSkycastersViasat Inc   
34530  021980002001051  HughesNetSkycastersAP&T Wireless, Inc.ALASKA C...   
44352  022900003001153                   HughesNetGCI Communication Corp.   
26694  021700004021037  HughesNetViasat IncMATANUSKA TELEPHONEGCI Comm...   
39783  022610003001134                      HughesNetSkycastersViasat Inc   
37228  022400001002024  HughesNetSkycastersAlaska Power & TelephoneGCI...   
39432  022610002001278  HughesNetSkycastersViasat IncCordova Telecom C...   
32802  021950002004183                                HughesNetSkycasters   
468    020130001002013                                          HughesNet   
36486  022400001001102         HughesNetSkycastersGCI Communication Corp.   
24     020130001001024                   HughesNetGCI Communication Corp.   
2227   020160002002203                              HughesNetOptimERA Inc   
12433  020900015003002  HughesNetGCI Communication Corp.ALASKA COMMUNI...   
35995  022200002001004  HughesNetSkycastersGCI Communication Corp.ALAS...   
12670  020900015006051  HughesNetGCI Communication Corp.GCI Communicat...   
41988  022820001001036         HughesNetSkycastersGCI Communication Corp.   
31888  021880001001139                   HughesNetGCI Communication Corp.   
24090  021640001002189                                          HughesNet   
45079  022900004002119         HughesNetViasat IncGCI Communication Corp.   
17833  021220002002141                                HughesNetViasat Inc   
25203  021700001011778         HughesNetSkycastersGCI Communication Corp.   
3468   020200006005005  HughesNetSkycastersViasat IncBorealis Broadban...   
6906   020500001002048  HughesNetGCI Communication Corp.GCI Communicat...   
12875  020900016004023  HughesNetAlasConnect, LLCAlasConnect, LLCGCI C...   
17962  021220002003029  HughesNetSPITwSPOTS, IncViasat IncALASKA COMMU...   
10941  020900008004013  HughesNetAlasConnect, LLCALASKA COMMUNICATIONS...   
4892   020200019004010  HughesNetSkycastersViasat IncBorealis Broadban...   
19758  021220008001087  HughesNetALASKA COMMUNICATIONSALASKA COMMUNICA...   
19777  021220008002001  HughesNetSPITwSPOTS, IncALASKA COMMUNICATIONSG...   
9153   020680001002470                   HughesNetGCI Communication Corp.   
19038  021220005004037  HughesNetSPITwSPOTS, IncViasat IncGCI Communic...   
29098  021800001001083  HughesNetMukluk Telephone Company, Inc.GCI Com...   
8747   020680001002064                   HughesNetGCI Communication Corp.   
40283  022610003001634                      HughesNetSkycastersViasat Inc   
34928  022200001001067                                HughesNetSkycasters   
38795  022610001002031                                HughesNetSkycasters   
14668  021000001001128                                HughesNetSkycasters   
26723  021700004021066                                HughesNetViasat Inc   
31776  021880001001027                   HughesNetGCI Communication Corp.   
28169  021700010011021  HughesNetSkycastersViasat IncMATANUSKA TELEPHO...   
550    020130001002095                                          HughesNet   
15155  021000001001615                                HughesNetSkycasters   
17234  021220001001135                                HughesNetViasat Inc   

      Pr